# Space-time Analytics Lab 3
## Becky Davies, Gabi Murillo, Adam Mahood, Barry Price, Johannes Uhl
## November 2016
### Analysis of Brown Bear trajectories in Slovenia, 1993-1999

#### Contents:
A. Introduction

- Maybe Barry can insert a paragraph here on bears in slovenia in general... B

B. Visualisation of the data using 2D maps

- Measurements per bear J
- Measurements per year J

C. Analysis of seasonal effects on movement speed

- Bear speed map J
- Gabi's results G

D. Home range and interaction analysis

- Kernel density maps for each bear, for each year J
- Examining interaction using wildlifeDI package A
- Examining interaction using space-time cubes B

E. Concluding remarks
- all?

Data Source: movebank.org





In [1]:
#loading libraries:

import pandas as pd
import json
from IPython.display import Javascript
from IPython.display import HTML
import numpy as np
import seaborn as sns; sns.set(color_codes=True)
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import animation
import random
import scipy.spatial.distance
import math
import shapely.geometry as sg
import geopandas as gp
%config InlineBackend.figure_format = 'svg'
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 16, 9
%matplotlib inline
%pylab inline

import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('johannesuhl', '1xhtdlbk6o')

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
#read the data
csvfile=r'./Data/Brown bear Slovenia 1993-1999UTM.csv'
df = pd.read_csv(csvfile) 
#make sure that you adjust your code to the pandas dataframe object df, we only want to read the data once.


### A. Introduction

### B. Visualisation of the data using 2D maps

### C. Analysis of seasonal effects on movement speed

### D. Home range and interaction analysis

The interaction analysis is done in R using wildlifeDI package.
The notebook requires a separate R kernel and can be accessed here:

https://github.com/johannesuhl/STA-lab3/blob/master/Wildlife%20Trajectories%20in%20R.ipynb



#### Visualizing Space-Time Cube for paths of all brown bears in study using Plotly

Using the Plotly visualization library we visualized the space-time paths of all bears in the study. The cube reveals that many of the bears, such as Srecko, were recorded over relatively short time intervals. Almost none span the entire study period. Furthermore, the cube conveys the concentration of bears in a particular area throughout time, with only a few bears wandering afield. The dispersion could reflect a concentration of bears in a particular study area or home range, or a concentration of the observers who measured the presence of the bears across time.

In [3]:
def brownian_motion(T = 1, N = 100, mu = 0.1, sigma = 0.01, S0 = 20):
    dt = float(T)/N
    t = np.linspace(0, T, N)
    W = np.random.standard_normal(size = N)
    W = np.cumsum(W)*np.sqrt(dt) # standard brownian motion
    X = (mu-0.5*sigma**2)*t + sigma*W
    S = S0*np.exp(X) # geometric brownian motion
    return S

In [4]:
data = []
for tag in set(df['tag-local-identifier']):
    bear = df[df['tag-local-identifier'] == tag]

    x = pd.Series(bear['utm-easting'])
    y = pd.Series(bear['utm-northing'])
    dates = pd.to_datetime(bear['timestamp'])

    trace = go.Scatter3d(
        x=x, y=y, z=dates,
        marker=dict(
            size=4,
            color="#%06x" % random.randint(0, 0xFFFFFF),
            colorscale='Viridis',
        ),
        line=dict(
            color = '#444ddd',
            width=1
        ),
        name = tag
    )

    data.append(trace)
    
layout = dict(
    width=800,
    height=700,
    autosize=False,
    title='Bears movement Slovenia',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        camera=dict(
            up=dict(
                x=0,
                y=0,
                z=1
            ),
            eye=dict(
                x=-1.7428,
                y=1.0707,
                z=0.7100,
            )
        ),
        aspectratio = dict( x=1, y=1, z=0.7 ),
        aspectmode = 'manual'
    ),
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='bears_slovenia_test', height=1200, validate=False)

#### Examining spatial-temporal interaction of bears between each other

By checking for nearness in space-time, we searched for potential interactions between bears. The code below can be modified to adjust the search radius in time and space. Using time slices of 24 hours (12 hours before and after a recorded position), and spatial radii of 100 m around each point, we gain an impression of the potential interactions that occurred. In many cases, a pair of bears consistently appears to interact, suggesting they have some sort of relationship, perhaps family members. For instance, Mishko and Ancka repeatedly appear within the same space-time window, as do Vera and Maja. In accordance with the temporal extent of each bear's points in the space-time cube, these interactions are limited across the temporal range of the study. Also, they tend to arise in the area with the highest concentration of points, an expected outcome given that where there is a higher concentration of bears we can expect a higher likelihood of bear interactions.


In [5]:
# create interval in which to search for nearness in time
# can be modified by adjusting window
def find_interval(x):
    start = np.datetime64(x) + np.timedelta64(-12, 'h')
    end = np.datetime64(x) + np.timedelta64(12, 'h')
    return start, end
# checks whether point is within time interval
def check_interval(x, start, end):
    x = np.datetime64(x)
    if start < x < end:
        return (start, end)
    else:
        return False

In [ ]:
# for each bear in data set
matches = pd.DataFrame()
all_geo = pd.DataFrame()
df.drop('event-id', axis=1, inplace=True)
df.drop_duplicates(inplace=True)
for bear in set(df['tag-local-identifier']):
    focal_bear = df[df['tag-local-identifier'] == bear]
    print bear
    # isolate points for single bear
    focal_bear = df[df['tag-local-identifier'] == bear]
    # from timestamps, find time interval
    focal_bear['interval'] = focal_bear.timestamp.apply(find_interval)
    #search all other data timestamps for presence in interval
    other_bears = df[df['tag-local-identifier'] != bear] 
    # for interval in focal bear
    for x in focal_bear.interval:
        # check for matches in rest of dataframe
        matches = pd.DataFrame()
        for y in other_bears.timestamp:
            # if match found in time
            if x[0] < np.datetime64(y) < x[1]:
                # create DF containing matching records
                matches = matches.append(other_bears[other_bears.timestamp == y])
                # save initial bear record
                record = focal_bear[focal_bear.interval == x]
                # create point from bear record
                point = sg.Point(record['utm-easting'], record['utm-northing'])
                # create buffer around point
                polygon = point.buffer(100)
                # search through matches for points within buffer polygon
                record.interval[0] = list(record.interval.values[0])
                record.drop('interval', axis=1, inplace=True)
                for i in range(len(matches)):
                    point_bear = sg.Point(matches['utm-easting'].values[i], matches['utm-northing'].values[i])
                    # if match found
                    if point_bear.within(polygon):
                        # add matching point to DF
                        dox = matches[(matches['utm-easting'] == matches['utm-easting'].values[i]) & (matches['utm-northing'] == matches['utm-northing'].values[i])]
                        dox = pd.concat([dox, record], join="outer")
                        crs = "+proj=utm +zone=33 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
                        geometry = [sg.Point(ab) for ab in zip(dox['utm-easting'], dox['utm-northing'])]
                        geo_df = gp.GeoDataFrame(dox, crs=crs, geometry=geometry)
                        geo_df.drop('visible', axis=1, inplace=True)
                        all_geo = pd.concat([all_geo, geo_df], join="outer")
                        geo_df.to_file(bear + '_' + str(i) + '.shp')
                    
                    else:
                        pass
            else: 
                pass
    try: 
        del record
        del geo_df
        del dox
    except:
        pass
all_geo.to_file('interactions.shp')
all_geo.plot()

### E. Concluding remarks